In [ ]:
import os
import cv2
import math
import shutil

# Configurações
BASE_DIR = "/content/yolo-dataset"
IMG_DIR = os.path.join(BASE_DIR, "images")
LABEL_DIR = os.path.join(BASE_DIR, "labels")

OUTPUT_DIR = "/content/yolo-dataset-sliced"
TILE_SIZE = 1024

SUBSETS = ["train", "val"]  # Apenas train e val

def read_labels(label_path):
    labels = []
    if not os.path.exists(label_path):
        return labels
    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            w = float(parts[3])
            h = float(parts[4])
            labels.append([cls, x_center, y_center, w, h])
    return labels

def save_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

def yolo_to_box(xc, yc, w, h, img_w, img_h):
    xmin = (xc - w / 2) * img_w
    xmax = (xc + w / 2) * img_w
    ymin = (yc - h / 2) * img_h
    ymax = (yc + h / 2) * img_h
    return xmin, ymin, xmax, ymax

def box_to_yolo(xmin, ymin, xmax, ymax, img_w, img_h):
    xc = (xmin + xmax) / 2 / img_w
    yc = (ymin + ymax) / 2 / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h
    return [xc, yc, w, h]

def process_subset(subset):
    img_subset_dir = os.path.join(IMG_DIR, subset)
    label_subset_dir = os.path.join(LABEL_DIR, subset)
    out_img_dir = os.path.join(OUTPUT_DIR, "images", subset)
    out_label_dir = os.path.join(OUTPUT_DIR, "labels", subset)

    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_label_dir, exist_ok=True)

    img_files = [f for f in os.listdir(img_subset_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for img_file in img_files:
        img_path = os.path.join(img_subset_dir, img_file)
        label_path = os.path.join(label_subset_dir, img_file.rsplit('.',1)[0] + ".txt")

        img = cv2.imread(img_path)
        if img is None:
            print(f"Erro ao ler imagem {img_path}")
            continue
        h_img, w_img = img.shape[:2]

        labels = read_labels(label_path)

        # Se a imagem for menor ou igual a TILE_SIZE em largura ou altura, copia direto
        if w_img <= TILE_SIZE or h_img <= TILE_SIZE:
            out_img_path = os.path.join(out_img_dir, img_file)
            out_label_path = os.path.join(out_label_dir, img_file.rsplit('.',1)[0] + ".txt")
            shutil.copy2(img_path, out_img_path)
            save_labels(out_label_path, labels)
            print(f"Copiado {img_file} sem fatiamento")
            continue

        # Quantidade de tiles horizontal e vertical
        tiles_x = math.ceil(w_img / TILE_SIZE)
        tiles_y = math.ceil(h_img / TILE_SIZE)

        for ty in range(tiles_y):
            for tx in range(tiles_x):
                x_start = tx * TILE_SIZE
                y_start = ty * TILE_SIZE
                x_end = min(x_start + TILE_SIZE, w_img)
                y_end = min(y_start + TILE_SIZE, h_img)

                tile_w = x_end - x_start
                tile_h = y_end - y_start

                tile_img = img[y_start:y_end, x_start:x_end]

                tile_labels = []
                for label in labels:
                    cls, xc, yc, bw, bh = label
                    xmin, ymin, xmax, ymax = yolo_to_box(xc, yc, bw, bh, w_img, h_img)

                    inter_xmin = max(xmin, x_start)
                    inter_ymin = max(ymin, y_start)
                    inter_xmax = min(xmax, x_end)
                    inter_ymax = min(ymax, y_end)

                    inter_w = inter_xmax - inter_xmin
                    inter_h = inter_ymax - inter_ymin

                    if inter_w <= 0 or inter_h <= 0:
                        continue

                    new_xmin = inter_xmin - x_start
                    new_ymin = inter_ymin - y_start
                    new_xmax = inter_xmax - x_start
                    new_ymax = inter_ymax - y_start

                    if new_xmax - new_xmin < 2 or new_ymax - new_ymin < 2:
                        continue

                    new_xc, new_yc, new_w, new_h = box_to_yolo(new_xmin, new_ymin, new_xmax, new_ymax, tile_w, tile_h)

                    tile_labels.append([cls, new_xc, new_yc, new_w, new_h])

                if len(tile_labels) > 0:
                    tile_img_name = f"{img_file.rsplit('.',1)[0]}_{ty}_{tx}.png"
                    tile_label_name = f"{img_file.rsplit('.',1)[0]}_{ty}_{tx}.txt"

                    cv2.imwrite(os.path.join(out_img_dir, tile_img_name), tile_img)
                    save_labels(os.path.join(out_label_dir, tile_label_name), tile_labels)

        print(f"Processado {img_file} - {tiles_y}x{tiles_x} tiles")

def main():
    for subset in SUBSETS:
        print(f"Processando {subset}...")
        process_subset(subset)
    print("Corte concluído!")

if __name__ == "__main__":
    main()